In [1]:
# https://docs.databricks.com/dev-tools/data-pipelines.html

!pip install "apache-airflow[databricks, celery, s3, password]"

In [2]:
!airflow db init

DB: sqlite:////Users/xheng/airflow/airflow.db
[2021-03-11 17:56:08,927] {db.py:674} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [unusual_prefix_fb93137681c49340e364f4c6da380f426bb276c8_example_kubernetes_executor_config] Could not import DAGs in example_kubernetes_executor_config.py: No module named 'kubernetes'
WARNI [unusual_prefix_fb93137681c49340e364f4c6da380f426bb276c8_example_kubernetes_executor_config] Install kubernetes dependencies with: pip install apache-airflow['cncf.kubernetes']
Initialization done


In [3]:
# 
import airflow
from airflow import DAG
from airflow.contrib.operators.databricks_operator import DatabricksSubmitRunOperator

args = {
    'owner': 'airflow',
    'email': ['airflow@example.com'],
    'depends_on_past': False,
    'start_date': airflow.utils.dates.days_ago(0)
}

dag = DAG(dag_id='example_databricks_operator', default_args=args, schedule_interval='@daily')

new_cluster = {
    'spark_version': '6.0.x-scala2.11',
    'node_type_id': 'i3.xlarge',
    'aws_attributes': {
        'availability': 'ON_DEMAND'
    },
    'num_workers': 8
}

notebook_task_params = {
    'new_cluster': new_cluster,
    'notebook_task': {
    'notebook_path': '/Users/airflow@example.com/PrepareData',
  },
}


In [4]:

# Example of using the JSON parameter to initialize the operator.
notebook_task = DatabricksSubmitRunOperator(
  task_id='notebook_task',
  dag=dag,
  json=notebook_task_params)

# Example of using the named parameters of DatabricksSubmitRunOperator to initialize the operator.
spark_jar_task = DatabricksSubmitRunOperator(
  task_id='spark_jar_task',
  dag=dag,
  new_cluster=new_cluster,
  spark_jar_task={
    'main_class_name': 'com.example.ProcessData'
  }
)

notebook_task.set_downstream(spark_jar_task)

In [11]:
!airflow version
!airflow dags list

2.0.1
dag_id                        | filepath                      | owner   | paused
==============================+===============================+=========+=======
example_bash_operator         | /Users/xheng/Applications/min | airflow | True  
                              | iconda3/envs/py3.7.4/lib/pyth |         |       
                              | on3.7/site-packages/airflow/e |         |       
                              | xample_dags/example_bash_oper |         |       
                              | ator.py                       |         |       
example_branch_dop_operator_v | /Users/xheng/Applications/min | airflow | True  
3                             | iconda3/envs/py3.7.4/lib/pyth |         |       
                              | on3.7/site-packages/airflow/e |         |       
                              | xample_dags/example_branch_py |         |       
                              | thon_dop_operator_3.py        |         |       
example_branch_operato